In [41]:
import pandas as pd
import os
from difflib import SequenceMatcher
from sklearn import preprocessing



In [42]:
lega = pd.read_csv('fanta_nmc.csv', sep=';')
for c in lega.columns:
    lega[c] = lega[c].str.lower()
lega = lega.rename(columns={'Unnamed: 0': 'ruolo'})
lega['ruolo'] = lega['ruolo'].str[0].str.upper()
lega = lega.dropna()


In [43]:
root = '../data/gazzetta_dello_sport_3plus/2020-21/'

dots =1
filename = '2020-21_dots_%d.csv'
stagione = pd.DataFrame()
while len(os.listdir(root+str(dots))) > 0 :
    stagione = pd.concat([
        stagione, pd.read_csv(root+str(dots)+'/'+filename%dots)
    ],
    axis=0)
    dots+=1
    
stagione = stagione.dropna()
stagione = stagione[stagione.ruolo!='0']

## || pr(Titolarità) * voto mediano ||

In [44]:
n_giornate = stagione.dots.max()
partite_giocate = stagione.groupby(['giocatore', 'ruolo', 'squadra']).count().sort_values('dots').div(0.01*n_giornate)
pr_tit = partite_giocate['dots']

fv_median = stagione.groupby(['giocatore', 'ruolo', 'squadra']).median().sort_values('fantavoto')
fv_median = fv_median['fantavoto']

fv_median_hat = stagione.groupby(['giocatore', 'ruolo', 'squadra']).median().sort_values('fantavoto').div(n_giornate)
fv_median_hat = fv_median_hat['fantavoto']

perf = pd.DataFrame(columns=['pr', 'fv'])
perf['pr'] = pr_tit
perf['fv'] = fv_median
perf['fv_hat'] = fv_median_hat
perf['metric'] = perf['pr'] * perf['fv_hat']



perf = perf.sort_values(by='giocatore', ascending=False)
perf = perf.reset_index()

## Filter stagione

In [59]:
def get_list_per_position(lega, position):
    players=lega[lega.ruolo == position].set_index('ruolo').values.tolist()
    players_expl = []
    for p in players:
        players_expl+=p
    return players_expl

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def index_to_filter(lista, perf, ruolo):
    n_deleted = 0
    deleted=[]
    for giocatore_in_lega in lista:
        for index, row in perf.iterrows():
            try:
                if giocatore_in_lega in str(row['giocatore']) or\
                similar(giocatore_in_lega, row['giocatore']) >= 0.70:
                    deleted.append(index)
                    break
            except TypeError:
                print(giocatore_in_lega, row['giocatore'])
    
    return deleted


portieri = get_list_per_position(lega, 'P')
difensori = get_list_per_position(lega, 'D')
centrocampisti = get_list_per_position(lega, 'C')
attaccanti = get_list_per_position(lega, 'A')


g2r=[]
g2r += index_to_filter(portieri, perf, 'P')
g2r += index_to_filter(difensori, perf, 'D')
g2r += index_to_filter(centrocampisti, perf, 'C')
g2r += index_to_filter(attaccanti, perf, 'A')


perf_f = perf.loc[set(perf.index) - set(g2r)]


In [60]:
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(perf_f[['metric']].values.astype(float))
perf_f['metric_norm'] = x_scaled

In [61]:
for ruolo in ['P', 'D', 'C', 'A']:
    tmp = perf_f[perf_f.ruolo==ruolo].sort_values(by='metric', ascending=False)
    tmp['q'] = pd.qcut(tmp['metric'], 4, labels=False) 
    
    tmp.to_csv(ruolo+'.csv', sep=';', decimal=",",index=False)

In [62]:
tmp

,giocatore,ruolo,squadra,pr,fv,fv_hat,metric,metric_norm,q
228,martinez l,A,inter,100.000000,7.00,0.368421,36.842105,1.000000,3
304,insigne l,A,napoli,73.684211,9.25,0.486842,35.872576,0.973684,3
212,messias j,A,crotone,100.000000,6.50,0.342105,34.210526,0.928571,3
183,nzola m,A,spezia,68.421053,9.00,0.473684,32.409972,0.879699,3
71,simy n,A,crotone,100.000000,6.00,0.315789,31.578947,0.857143,3
172,palacio r,A,bologna,94.736842,6.00,0.315789,29.916898,0.812030,3
306,improta r,A,benevento,89.473684,6.00,0.315789,28.254848,0.766917,3
283,kouame c,A,fiorentina,73.684211,6.00,0.315789,23.268698,0.631579,3
436,colley e,A,verona,73.684211,6.00,0.315789,23.268698,0.631579,3
134,politano m,A,napoli,73.684211,6.00,0.315789,23.268698,0.631579,3


In [ ]:
test = perf.reset_index()
test[test.giocatore.str.contains('ronaldo')]